# N.B.
- [ ] Create a class to save data into a file
- [ ] Check pauses into the codes and remove redundant ones (or add them if necessary).
- [ ] With sGen.pul_trig_mode(**'AUTO'**) there aren't errors, instead with sGen.pul_trig_mode(**'SING'**) (theorically more correct) the code stops working.  
    *-1074118634: The trigger has not occurred for the requested buffer, so data cannot be fetched relative to the trigger.*
- [ ] controllare se ad alte frequenze se le oscillazioni si vedono anche sull'oscilloscopio
- [ ] CHIEDERE RIGUARDO AI BATTIMENTI (ES SWEEP 39)



In [4]:
# Uscita trigger dal signal generator: Pulse Sync

%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

sys.path.insert(1, '..\\..\\Instruments\\network\\RS_Signal_Generator')
sys.path.insert(2, '..\\..\\Instruments\\Acquisition_Card')
sys.path.insert(3, '..\\..\\Instruments\\QuickSyn')
sys.path.insert(4, '..\\..\\HDF5')

import RS_SMA100B
import FSL_0010
from Card_Class import PXIe5170R
from HDF5 import *

In [5]:
QSyn_port       = 'COM35'
SG_ip           = '192.168.40.15'                                   # Set IP address of the device
Card_Name       = 'PXI1Slot3'

fsl             = FSL_0010.FSL10_synthesizer(device_address = QSyn_port)
sGen            = RS_SMA100B.SMA100B(ip = SG_ip)
pxie            = PXIe5170R(resource_name = Card_Name)

FSL_0010:	Connection successful!
SMA100B:	Connection successful!
PXIe_5170R:    Connection successful!


In [31]:
pulse_f_min     = 2.010e9
# pulse_f_max		= 

amplitude		    = -6                                               # Set the amplitude of the signal in dBm
# amp_max			= 


sample_rate     = 250e6                                             # Maximum Value: 250.0e6
k = 4                                                               # coefficiente per prendere k*1000 punti 
																	# con la scheda, dunque devo aumentare la pulse width e period
pulse_period    = k * 1e-6                                          # 4e-6 con 250e6 dà 1000 punti
num_pts         = int(sample_rate * pulse_period)                   # min 5ns                             
pulse_delay     = 0

percent         = 5
pulse_width     = pulse_period * (1 - percent/100)                  # min 20ns


sGen.reset()
sGen.clear()
sGen.pul_gen_params(delay = pulse_delay, width = pulse_width, period = pulse_period)   # da capire quale pulse width
sGen.pul_gen_mode('SING')
sGen.pul_trig_mode('SING')


# parameters for the Acquisition Card
pxie.voltage_range      = 0.5
pxie.coupling           = 'AC' # or DC????
pxie.num_pts            = num_pts
pxie.num_records        = 1
pxie.ref_position       = 0
pxie.sample_rate        = sample_rate                               # Maximum Value: 250.0e6


# FSL-0010
fsl_freq = 2
fsl.set_frequency(fsl_freq) # GHz

'FSL_0010: Frequency set to 2 GHz.'

In [32]:
trig_CH = 3
I_CH	= 2
Q_CH	= 0


fsl.set_output('ON')
pxie.open(trigger_channel = str(trig_CH))

pula        = np.arange(pulse_f_min, pulse_f_min + 0.010e9, 0.001e9)

data_dict   = {'power_(dBm)': amplitude, 'freqs': {}}
counter     = 1

digits_f = "{:0"+str(len(str(len(pula))))+"d}"
sGen.RF_lvl_ampl(amplitude)

for i, pul in enumerate(pula):
	
	sGen.RF_freq(pul) 
	sGen.pul_state(1)
	sGen.RF_state(1)
	waveforms = pxie.acquisition(sGen.pul_exe_sing_trig)
	I = np.array(waveforms[I_CH].samples.tolist())
	Q = np.array(waveforms[Q_CH].samples.tolist())
	# if p == 0 and i == 0:

	# 	cln_pts = 30                # clean points for I-Q plane
	# 	plt.clf()
	# 	plt.scatter(Q[cln_pts:-cln_pts],I[cln_pts:-cln_pts])
	# 	plt.show()
	
	
	sGen.pul_state(0)
	sGen.RF_state(0)

	print(f'\rf{digits_f.format(i)}	: {int(counter*100/len(pula))} %', end='')
	sys.stdout.flush()
	counter += 1
	data_dict['freqs'][f'f{digits_f.format(i)}'] = {'freq_(Hz)': pul, 'I': I, 'Q': Q}
fsl.set_output('OFF')

f09	: 100 %

'RF set to OFF'

In [33]:
# SAVE DATA ON HDF5 FILE
filename = f'Mixer_1135_{fsl_freq}GHz.h5'
if os.path.exists(filename):
  os.remove(filename)
save_dict_to_hdf5(data_dict, filename)

## mLa scelta del tappo influiva leggermente sull'ampiezza letta.

In [2]:
import Tools as T

T.mV_to_dBm(31.622776601683793)

-20.0